In [47]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [48]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

## 默認Config檔案在data/config.py內

In [49]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [50]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [51]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [52]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [53]:
PriorBox_Demo=PriorBox(cfg)

In [54]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [55]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [56]:
Use_cuda=True
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [57]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [58]:
'''要不要使用gpu'''
Use_cuda=True

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [59]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [60]:
epochs=300
iteration=1000

In [61]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Weights.pth')

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()


BBOX Regression Loss:  2.512907211868851
Classification Loss:  14.194819132486979
BBOX Regression Loss:  2.512758735374168
Classification Loss:  14.080324978298615
BBOX Regression Loss:  2.5126106262207033
Classification Loss:  13.966558046694155
BBOX Regression Loss:  2.512462799637406
Classification Loss:  13.858390751591434
BBOX Regression Loss:  2.5123152273672598
Classification Loss:  13.762608800817418
BBOX Regression Loss:  2.5121677257396557
Classification Loss:  13.674130211347414
BBOX Regression Loss:  2.5120201998917517
Classification Loss:  13.590070710358797
BBOX Regression Loss:  2.5118726059242533
Classification Loss:  13.508882776896161
BBOX Regression Loss:  2.511725001080046
Classification Loss:  13.430130896548672
BBOX Regression Loss:  2.5115773631908276
Classification Loss:  13.355104641384548
BBOX Regression Loss:  2.5114303331985224
Classification Loss:  13.284896532052294
BBOX Regression Loss:  2.511286615442347
Classification Loss:  13.220743862199194
BBOX Regr

BBOX Regression Loss:  2.5006726229632346
Classification Loss:  12.30924420844184
BBOX Regression Loss:  2.4850648668077255
Classification Loss:  12.139620971679687
BBOX Regression Loss:  2.484815060650861
Classification Loss:  12.139115679705585
BBOX Regression Loss:  2.4845624193733125
Classification Loss:  12.138605791256747
BBOX Regression Loss:  2.484306582698116
Classification Loss:  12.138091447618274
BBOX Regression Loss:  2.484046913429543
Classification Loss:  12.137571411132814
BBOX Regression Loss:  2.483782991950895
Classification Loss:  12.137044798297644
BBOX Regression Loss:  2.483514396223441
Classification Loss:  12.136511004412617
BBOX Regression Loss:  2.4832408834386754
Classification Loss:  12.135969345657916
BBOX Regression Loss:  2.4829623469599973
Classification Loss:  12.13541963051376
BBOX Regression Loss:  2.4826783752441406
Classification Loss:  12.134861947518809
BBOX Regression Loss:  2.4823884392426874
Classification Loss:  12.134298192290748
BBOX Regres

Classification Loss:  11.822328856782523
BBOX Regression Loss:  2.294910860556143
Classification Loss:  11.811271239104094
BBOX Regression Loss:  1.8590071784125435
Classification Loss:  10.68495551215278
BBOX Regression Loss:  1.8552719116210938
Classification Loss:  10.669177641691986
BBOX Regression Loss:  1.851540939896195
Classification Loss:  10.653344048394098
BBOX Regression Loss:  1.8478169900399668
Classification Loss:  10.63744413587782
BBOX Regression Loss:  1.8441019609239369
Classification Loss:  10.621442373770256
BBOX Regression Loss:  1.8404050568003711
Classification Loss:  10.605304369514373
BBOX Regression Loss:  1.8367154419106786
Classification Loss:  10.589008456437044
BBOX Regression Loss:  1.8330267199763548
Classification Loss:  10.572544267442492
BBOX Regression Loss:  1.8293382322837295
Classification Loss:  10.555897390106576
BBOX Regression Loss:  1.8256528642442496
Classification Loss:  10.539044833713106
BBOX Regression Loss:  1.8219718059706769
Classifi

BBOX Regression Loss:  1.3021591297125399
Classification Loss:  7.0319283699124115
BBOX Regression Loss:  1.2929675394694011
Classification Loss:  6.997851963184497
BBOX Regression Loss:  0.3728784101980704
Classification Loss:  3.589685906304253
BBOX Regression Loss:  0.36821261865121346
Classification Loss:  3.572650386669017
BBOX Regression Loss:  0.3636776829943245
Classification Loss:  3.5558624738528404
BBOX Regression Loss:  0.359455817717093
Classification Loss:  3.539097679985894
BBOX Regression Loss:  0.3554211828443739
Classification Loss:  3.5224538562915946
BBOX Regression Loss:  0.3514768847712764
Classification Loss:  3.506006490448374
BBOX Regression Loss:  0.3476368202734246
Classification Loss:  3.4896724196338154
BBOX Regression Loss:  0.34390926670145106
Classification Loss:  3.4734009742736816
BBOX Regression Loss:  0.3402667759867853
Classification Loss:  3.457171731328768
BBOX Regression Loss:  0.3367134959609421
Classification Loss:  3.4409540077492045
BBOX Regr

...

BBOX Regression Loss:  0.04441052612330428
Classification Loss:  0.0
BBOX Regression Loss:  0.04392188634143196
Classification Loss:  0.0
BBOX Regression Loss:  0.04335695622255904
Classification Loss:  0.0
BBOX Regression Loss:  0.04277745851866734
Classification Loss:  0.0
BBOX Regression Loss:  0.042200629978123176
Classification Loss:  0.0
BBOX Regression Loss:  0.04163476021910735
Classification Loss:  0.0
BBOX Regression Loss:  0.04108198131700219
Classification Loss:  0.0
BBOX Regression Loss:  0.040542975615471294
Classification Loss:  0.0
BBOX Regression Loss:  0.04001762702315911
Classification Loss:  0.0
BBOX Regression Loss:  0.039505627521980155
Classification Loss:  0.0
BBOX Regression Loss:  0.039006528364380046
Classification Loss:  0.0
BBOX Regression Loss:  0.03851986380573146
Classification Loss:  0.0
BBOX Regression Loss:  0.03804520089229314
Classification Loss:  0.0
BBOX Regression Loss:  0.037582091236264714
Classification Loss:  0.0
BBOX Regression Loss:  0.0371

BBOX Regression Loss:  0.06575076931903377
Classification Loss:  0.0
BBOX Regression Loss:  0.06513326467093832
Classification Loss:  0.0
BBOX Regression Loss:  0.06445341174583959
Classification Loss:  0.0
BBOX Regression Loss:  0.063758911396647
Classification Loss:  0.0
BBOX Regression Loss:  0.06306532367725247
Classification Loss:  0.0
BBOX Regression Loss:  0.0623847032976753
Classification Loss:  0.0
BBOX Regression Loss:  0.06171814331901492
Classification Loss:  0.0
BBOX Regression Loss:  0.06116431109205224
Classification Loss:  0.0
BBOX Regression Loss:  0.06174180898602079
Classification Loss:  0.0
BBOX Regression Loss:  0.061355479336374245
Classification Loss:  0.0
BBOX Regression Loss:  0.06084198188396728
Classification Loss:  0.0
BBOX Regression Loss:  0.060258804691136404
Classification Loss:  0.0
BBOX Regression Loss:  0.059662829611836825
Classification Loss:  0.0
BBOX Regression Loss:  0.059073474734512806
Classification Loss:  0.0
BBOX Regression Loss:  0.00720614

BBOX Regression Loss:  0.25581941251401547
Classification Loss:  0.0
BBOX Regression Loss:  0.2534171507682329
Classification Loss:  0.0
BBOX Regression Loss:  0.25283758741207224
Classification Loss:  0.0
BBOX Regression Loss:  0.24836776400053945
Classification Loss:  0.0
BBOX Regression Loss:  0.24617321147840213
Classification Loss:  0.0
BBOX Regression Loss:  0.244634033397392
Classification Loss:  0.0
BBOX Regression Loss:  0.2354953567716936
Classification Loss:  0.0
BBOX Regression Loss:  0.221160549493679
Classification Loss:  0.0
BBOX Regression Loss:  0.20573747770468098
Classification Loss:  0.0
BBOX Regression Loss:  0.19160243805187446
Classification Loss:  0.0
BBOX Regression Loss:  0.17901188103390145
Classification Loss:  0.0
BBOX Regression Loss:  0.16789764962611825
Classification Loss:  0.0
BBOX Regression Loss:  0.15804707310436406
Classification Loss:  0.0
BBOX Regression Loss:  0.14927942448433626
Classification Loss:  0.0
BBOX Regression Loss:  0.141431467766468

Classification Loss:  0.0
BBOX Regression Loss:  0.10279315540503803
Classification Loss:  0.0
BBOX Regression Loss:  0.09868436307232413
Classification Loss:  0.0
BBOX Regression Loss:  0.09489162486831611
Classification Loss:  0.0
BBOX Regression Loss:  0.09137985292662111
Classification Loss:  0.0
BBOX Regression Loss:  0.08811894482223098
Classification Loss:  0.0
BBOX Regression Loss:  0.08508294986780689
Classification Loss:  0.0
BBOX Regression Loss:  0.08224939787761815
Classification Loss:  0.0
BBOX Regression Loss:  0.07959866498808656
Classification Loss:  0.0
BBOX Regression Loss:  0.07711362794169098
Classification Loss:  0.0
BBOX Regression Loss:  0.07477920595024724
Classification Loss:  0.0
BBOX Regression Loss:  0.07258213248596149
Classification Loss:  0.0
BBOX Regression Loss:  0.07051062275664191
Classification Loss:  0.0
BBOX Regression Loss:  0.06855420045193655
Classification Loss:  0.0
BBOX Regression Loss:  0.06670355707898995
Classification Loss:  0.0
BBOX Reg

BBOX Regression Loss:  0.06552840784292487
Classification Loss:  0.0
BBOX Regression Loss:  0.06403961597042782
Classification Loss:  0.0
BBOX Regression Loss:  0.06259863705140882
Classification Loss:  0.0
BBOX Regression Loss:  0.06121360982857346
Classification Loss:  0.0
BBOX Regression Loss:  0.05988615998523438
Classification Loss:  0.0
BBOX Regression Loss:  0.058614197496401454
Classification Loss:  0.0
BBOX Regression Loss:  0.057394813958501205
Classification Loss:  0.0
BBOX Regression Loss:  0.056225090913813516
Classification Loss:  0.0
BBOX Regression Loss:  0.05510211092558758
Classification Loss:  0.0
BBOX Regression Loss:  0.054023112958463175
Classification Loss:  0.0
BBOX Regression Loss:  0.05298559425323751
Classification Loss:  0.0
BBOX Regression Loss:  0.05198721822658242
Classification Loss:  0.0
BBOX Regression Loss:  0.05102581127549283
Classification Loss:  0.0
BBOX Regression Loss:  0.05009936599157318
Classification Loss:  0.0
BBOX Regression Loss:  0.04920

In [ ]:
#loc_data, conf_data, priors = out
#print(loc_data.shape)             torch.Size([4, 8732, 4])
#print(conf_data.shape)            torch.Size([4, 8732, 21])
#print(priors.shape)               torch.Size([8732, 4])
#print(y[0].shape)                 torch.Size([4, 5])

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch